In [1]:
## Basic stuff
%load_ext autoreload
%autoreload

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
display(HTML("""<style>div.output_area{max-height:10000px;overflow:scroll;}</style>"""))
#IPython.Cell.options_default.cm_config.lineNumbers = true;

In [2]:
from fileIO import fileIO
from fileInfo import fileInfo
from pandas import read_csv, DataFrame, Series
from timeUtils import timestat
from numpy import nan

from glob import glob
from fileIO import csvIO
io = csvIO()

def loadData(ifile):
    mbdata = io.get(ifile, delimiter="\t", header=None)
    #mbdata = read_csv(ifile, delimiter="\t", header=None)
    mbdata = mbdata.replace('\\N', nan)
    return mbdata

aIDs={"ArianaGrande": 823336, "BuddyHolly": 10937, "Rupaul": 34318, "U2": 197, "DMB": 502, "Bono": 35575}

# DB Data

In [ ]:
lengthData = open("flength.csv").readlines()
lengthData = [x.replace("\n", "").strip().split() for x in lengthData]
lengthData = Series({item[1].split("/")[1]: int(item[0]) for item in lengthData if len(item[1].split("/")) > 1})
lengthData

In [ ]:
list(lengthData.index)

In [ ]:
#lengthData['release_label']
lengthData[(lengthData <= 6)]

# Lookup

In [ ]:
colnames = {}
colnames["l_label_release"] = {0: "Index", 1: "NA1", 2: "NA2", 3: "ReleaseID"}

ts = timestat("Loading Area Data")
files = glob("mbdump/l_label_release")
lookupData = {fileInfo(ifile).basename: loadData(ifile) for ifile in files}
lookupData = {key: val[list(colnames[key].keys())].rename(columns=colnames[key]) for key,val in lookupData.items()} if colnames is not None else lookupData
print("Keys: {0}".format(lookupData.keys()))
ts.stop()

In [ ]:
lookupData['l_label_release'][lookupData['l_label_release']['ReleaseID'].isin(releaseIDs)]

In [ ]:
lookupData['l_label_release'].max()

# Area

In [ ]:
colnames = {}
colnames["area_type"] = {0: "AreaTypeID", 1: "AreaTypeName", 3: "AlsoAreaTypeID", 4: "AreaTypeDescr", 5: "AreaTypeUUID"}
colnames["area"]      = {0: "AreaID", 1: "AreaUUID", 2: "AreaName", 3: "AreaTypeID"}
colnames["area_gid_redirect"] = {0: "AreaGIDUUID", 1: "AreaGID"}
colnames["area_alias_type"] = {0: "AreaAliasTypeID", 1: "AreaAliasTypeName", 5: "AreaAliasTypeUUID"}
colnames["area_alias"] = {0: "AreaAliasID", 1: "NA1", 2: "AreaAlias", 3: "AreaLang", 6: "AliasTypeID", 7: "AreaSortName"}

ts = timestat("Loading Area Data")
files = glob("mbdump/area*")
areaData = {fileInfo(ifile).basename: loadData(ifile) for ifile in files}
print("Keys: {0}".format(areaData.keys()))
areaData = {key: val[list(colnames[key].keys())].rename(columns=colnames[key]) for key,val in areaData.items()} if colnames is not None else areaData
print("Keys: {0}".format(areaData.keys()))
ts.stop()

# Event

In [ ]:
colnames = {}
colnames["event_type"] = {0: "EventTypeID", 1: "EventTypeName", 4: "EventTypeDescr"}
colnames["event_alias_type"] = {0: "EventAliasTypeID", 1: "EventAliasTypeName", 5: "EventAliasTypeGID"}
colnames["event_alias"] = {0: "EventAliasID", 1: "EventID", 2: "EventAliasName", 3: "EventAliasLang", 7: "EventAliasName2"}
colnames["event"] = {0: "EventID", 1: "EventGID", 2: "EventName", 
                     3: "EventStartYear", 4: "EventStartMonth", 5: "EventStartDay", 6: "EventEndYear", 7: "EventEndMonth", 8: "EventEndDay"}

ts = timestat("Loading Event Data")
files = glob("mbdump/event*")
eventData = {fileInfo(ifile).basename: loadData(ifile) for ifile in files}
print("Keys: {0}".format(eventData.keys()))
eventData = {key: val[list(colnames[key].keys())].rename(columns=colnames[key]) for key,val in eventData.items() if key in colnames} if colnames is not None else eventData
print("Keys: {0}".format(eventData.keys()))
ts.stop()

In [ ]:
eventData['event']

# ISRC

In [ ]:
colnames = {}
colnames['isrc'] = {0: "ISRCID", 1: "RecordingID", 2: "ISRC"}
colnames['iswc'] = {0: "ISWCID", 1: "WorkID", 2: "ISWC"}
colnames['iso_3166_1'] = {0: "ISO31661ID", 1: "ISO31661"}
colnames['iso_3166_2'] = {0: "ISO31662ID", 1: "ISO31662"}
colnames['iso_3166_3'] = {0: "ISO31663ID", 1: "ISO31663"}

ts = timestat("Loading i* Code Data")
files = glob("mbdump/is*")
isrcData = {fileInfo(ifile).basename: loadData(ifile) for ifile in files}
print("Keys: {0}".format(isrcData.keys()))
isrcData = {key: val[list(colnames[key].keys())].rename(columns=colnames[key]) for key,val in isrcData.items()} if colnames is not None else isrcData
print("Keys: {0}".format(isrcData.keys()))
ts.stop()

# Medium

In [ ]:
colnames = {}
colnames["medium_format"] = {0: "MediumFormatID", 1: "MediumName", 2: "MediumGroupID", 3: "NA3", 4: "MediumIntroYear", 5: "MediumDescr", 6: "MediumGID"}
#colnames["medium_cdtoc"]  = {0: "NA0", 1: "NA1", 2: "NA2", 3: "NA3"}
colnames["medium"] = {0: "ReleaseID_1", 1: "ReleaseID", 2: "SideNum", 3: "MediumFormatID", 4: "NA4", 5: "NA5", 7: "NumTracks"}

ts = timestat("Loading Medium Data")
files = glob("mbdump/medium*")
mediumData = {fileInfo(ifile).basename: loadData(ifile) for ifile in files}
print("Keys: {0}".format(mediumData.keys()))
mediumData = {key: val[list(colnames[key].keys())].rename(columns=colnames[key]) for key,val in mediumData.items() if key in colnames} if colnames is not None else mediumData
print("Keys: {0}".format(mediumData.keys()))
ts.stop()

In [ ]:
mediumData['medium_format'].nunique()

In [ ]:
mediumData['medium'].nunique()

In [ ]:
## 12" Vinyl ; ReleaseID=1310220
#mediumData['medium'][mediumData['medium'].eq(1310220).any(1)]
# MediumID  ReleaseID  Sides?  Format?   ?
# 1310220   1274390    1       33        4
# 1351220   1310220    1       31        11      <- Matches https://musicbrainz.org/release/6b5f33a8-fc5e-4c1e-b379-c659ce20a1c8


## Digital Media : ReleaseID=1792210
#mediumData['medium'][mediumData['medium'].eq(1792210).any(1)]
# MediumID  ReleaseID  Sides?  Format?   ?
# 1792210   1694923    1      12	NaN	0	2015-11-30 00:32:22.335038+00	13
# 1904010   1792210    1      12	NaN	0	2016-06-20 18:41:01.106192+00	11   Both Match


# 2x12" Vinyl ; ReleaseID=1680415

#mediumData['medium'][mediumData['medium'].eq(1680415).any(1)]
# MediumID  ReleaseID  Sides?  Format?   ?
# 1680415	1598741	1	1	NaN	0	2015-04-27 04:37:49.277705+00	11
# 1775088	1680415	1	31	NaN	0	2015-10-31 12:02:35.843886+00	5
# 1775089	1680415	2	31	NaN	0	2015-10-31 12:02:35.843886+00	6

# 1792210   1694923    1      12	NaN	0	2015-11-30 00:32:22.335038+00	13
# 1904010   1792210    1      12	NaN	0	2016-06-20 18:41:01.106192+00	11   Both Match

# Pixies Velouria CD w/ 4 Tracks
# 3099, 1162482, 3097, 2259927

## CD
# mediumData['medium'][mediumData['medium'].eq(3099).any(1)]
# 3099	3099	1	1	NaN	0	2011-05-16 14:57:06.530063+00	4

## CD (Status=Promotional)
# mediumData['medium'][mediumData['medium'].eq(1162482).any(1)]
# 1162482	1146184	1	12	NaN	0	2012-04-14 05:35:56.931961+00	1
# 1181517	1162482	1	1	NaN	0	2012-05-30 00:53:24.512335+00	4

## CD
# mediumData['medium'][mediumData['medium'].eq(3097).any(1)]
# 3097	3097	1	1	NaN	0	2012-10-18 19:49:17.567219+00	4

## 12" Vinyl
# mediumData['medium'][mediumData['medium'].eq(2259927).any(1)]
# 2259927	2099781	1	12	NaN	0	2018-01-11 08:03:13.107915+00	5
# 2441117	2259927	1	31	NaN	0	2018-09-28 23:20:29.279923+00	4  <-- This matches Web

In [ ]:
mediumData['medium'].nunique()

In [ ]:
mediumData['medium'].shape

In [ ]:
mediumData['medium_cdtoc'][mediumData['medium_cdtoc'].eq(1598741).any(1)]

In [ ]:
mediumData['medium_format'][mediumData['medium_format'].eq(38).any(1)]

In [ ]:
mediumData['medium_format'][mediumData['medium_format'].eq('38').any(1)]

In [ ]:
mediumData['medium_format'][mediumData['medium_format'].eq(63).any(1)]

In [ ]:
mediumData['medium_format'][mediumData['medium_format'].eq(64).any(1)]

# Language

In [ ]:
colnames = {}
colnames["language"] = {0: "LanguageID", 1: "LanguageShort1", 2: "LanguageShort2", 3: "LanguageShort3", 4: "LanguageName", 5: "NA5", 6: "LanguageShort"}

ts = timestat("Loading URL Data")
files = glob("mbdump/language*")
languageData = {fileInfo(ifile).basename: loadData(ifile) for ifile in files}
print("Keys: {0}".format(languageData.keys()))
languageData = {key: val[list(colnames[key].keys())].rename(columns=colnames[key]) for key,val in languageData.items() if key in colnames} if colnames is not None else languageData
print("Keys: {0}".format(languageData.keys()))
ts.stop()

In [ ]:
languageData['language'][languageData['language']["LanguageID"] == 120]

# URL

In [ ]:
colnames = {}
colnames["url"] = {0: "URLID", 1: "URLGID", 2: "URLName"}
#colnames["url_gid_redirect"] = {0: "URLGIDUUID", 1: "URLGIDID"}

ts = timestat("Loading URL Data")
files = glob("mbdump/url*")
urlData = {fileInfo(ifile).basename: loadData(ifile) for ifile in files}
print("Keys: {0}".format(urlData.keys()))
urlData = {key: val[list(colnames[key].keys())].rename(columns=colnames[key]) for key,val in urlData.items() if key in colnames} if colnames is not None else urlData
print("Keys: {0}".format(urlData.keys()))
ts.stop()

# Script

In [ ]:
colnames = {}
colnames["script"] = {0: "ScriptID", 1: "ScriptName", 2: "NA2", 3: "ScriptDescr", 4: "NA4"}

ts = timestat("Loading URL Data")
files = glob("mbdump/script")
scriptData = {fileInfo(ifile).basename: loadData(ifile) for ifile in files}
scriptData = {key: val[list(colnames[key].keys())].rename(columns=colnames[key]) for key,val in scriptData.items() if key in colnames} if colnames is not None else scriptData
print("Keys: {0}".format(scriptData.keys()))
ts.stop()

# Work

In [10]:
colnames = {}
colnames["work_type"] = {0: "WorkTypeID", 1: "WorkTypeName", 3: "NA3", 4: "WorkTypeDescr", 5: "WorkTypeGID"}
#colnames["work_alias"] = {0: "WorkAliasID", 1: "WorkID", 2: "WorkName", 3: "WorkLang", 7: "WorkName2"}
#colnames["work_alias_type"] = {0: "WorkAliasTypeID", 1: "WorkAliasTypeName", 5: "WorkAliasTypeGID"}
colnames["work_attribute_type"] = {0: "WorkAttributeTypeID", 1: "WorkAttributeTypeName", 6: "WorkAttributeTypeDescr"}
colnames["work_attribute"] = {0: "WorkAttributeID", 1: "WorkID", 2: "WorkAttributeTypeID", 3: "NA3", 4: "WorkAttributeCode"}
colnames["work_language"] = {0: "WorkID", 1: "LanguageID"}
colnames["work"] = {0: "WorkID", 1: "WorkGID", 2: "WorkName", 3: "WorkTypeID"} #, 4: "WorkDescr"}

ts = timestat("Loading Work Data")
files = glob("mbdump/work*")
print(files)
workData = {fileInfo(ifile).basename: loadData(ifile) for ifile in files}
print("Keys: {0}".format(workData.keys()))
workData = {key: val[list(colnames[key].keys())].rename(columns=colnames[key]) for key,val in workData.items() if key in colnames} if colnames is not None else workData
print("Keys: {0}".format(workData.keys()))
ts.stop()

Current Time is Wed Oct 20, 2021 21:09 for Loading Work Data
['mbdump/work_type', 'mbdump/work_alias', 'mbdump/work_language', 'mbdump/work', 'mbdump/work_alias_type', 'mbdump/work_attribute_type_allowed_value', 'mbdump/work_attribute', 'mbdump/work_attribute_type']
Keys: dict_keys(['work_type', 'work_alias', 'work_language', 'work', 'work_alias_type', 'work_attribute_type_allowed_value', 'work_attribute', 'work_attribute_type'])
Keys: dict_keys(['work_type', 'work_language', 'work', 'work_attribute', 'work_attribute_type'])
Process [Loading Work Data] Took 11.9 Seconds


In [12]:
workData["work"]

,WorkID,WorkGID,WorkName,WorkTypeID,WorkDescr
0,12431434,86c6dd66-0da8-4b0f-8b78-ec9d4f12c5c4,Here to Go,NaN,NaN
1,12433103,712ff9ea-20d2-4c1a-aab7-6e4b7df9b4b2,1 John 4:16,17,The Mountain Goats song
2,12610029,ff2ebf6a-338d-468f-a0dd-ee5f489a962d,Goddess (of the Sad Man),17,NaN
3,12433476,567f5b20-909f-4388-8561-e622cdc1333c,來生緣,17,NaN
4,13596839,b0ba1f13-2c6d-49e5-a57d-9ec96f26eaa8,無限大ILLUSION 2017,17,NaN
5,8857245,85b243ac-66cb-3d58-b329-c40bcf088abb,"Overture to ""Funny Face""",NaN,NaN
6,9709654,577dcc1d-1838-3d53-9378-c30351f580c8,愛すべき未来へ,17,NaN
7,12610030,69cd3461-089e-4138-adc2-f3a1907a5013,The Night Is Over,17,NaN
8,13496641,1dbe7108-ba37-411d-a3af-2c2305a37c89,Als het effe kan,17,NaN
9,12433866,c3f6668f-70a9-424a-9639-488c7fbf7542,Fareweel Regality,17,NaN


In [ ]:
workData['work'][workData['work']["WorkGID"] == 'b8ee567b-0f09-3345-b794-b6279594aa29']

In [ ]:
for key,df in workData.items():
    print(key)
    print(df[df.eq(213928).any(1)])

# Label

In [ ]:
colnames = {}
colnames["label_alias"] = {0: "LabelAliasID", 1: "LabelID", 2: "LabelAliasName", 7: "LabelAliasName2"}
colnames["label_alias_type"] = {0: "LabelAliasTypeID", 1: "LabelAliasTypeName", 5: "LabelAliasGID"}
colnames["label_ipi"] = {0: "LabelIPIID", 1: "LabelIPI"}
colnames["label_isni"] = {0: "LabelISNIID", 1: "LabelISNI"}
colnames["label_type"] = {0: "LabelTypeID", 1: "LabelTypeName", 5: "LabelTypeGID"}
colnames["label"] = {0: "LabelID", 1: "LabelGID", 2: "LabelName"}

ts = timestat("Loading Label Data")
files = glob("mbdump/label*")
labelData = {fileInfo(ifile).basename: loadData(ifile) for ifile in files}
print("Keys: {0}".format(labelData.keys()))
labelData = {key: val[list(colnames[key].keys())].rename(columns=colnames[key]) for key,val in labelData.items() if key in colnames} if colnames is not None else labelData
print("Keys: {0}".format(labelData.keys()))
ts.stop()

# Recording

In [ ]:
colnames = {}
colnames["recording_alias_type"] = {0: "RecordingAliasTypeID", 1: "RecordingAliasTypeName"}
colnames["recording_alias"] = {0: "RecordingAliasID", 1: "RecordingID", 2: "RecordingAliasName", 3: "RecordingAliasLang", 7: "recordingAliasName2"}
colnames["recording"] = {0: "RecordingID", 1: "RecordingGID", 2: "RecordingName", 3: "ArtistID", 4: "TimeLength"} #, 5: "RecordingDescr"}

ts = timestat("Loading Recording Data")
files = glob("mbdump/rec")
recordingData = {fileInfo(ifile).basename: loadData(ifile) for ifile in files}
print("Keys: {0}".format(recordingData.keys()))
recordingData = {key: val[list(colnames[key].keys())].rename(columns=colnames[key]) for key,val in recordingData.items() if key in colnames} if colnames is not None else recordingData
print("Keys: {0}".format(recordingData.keys()))
ts.stop()

# Track

In [ ]:
colnames = {}
colnames["track"] = {0: "TrackID", 1: "TrackGID", 2: "RecordingID", 3: "NA3", 4: "TrackNum", 5: "TrackNumName", 6: "TrackName", 7: "ArtistID", 8: "TimeLength"}

# Release
# 2373946	7c5d14b4-cf40-4eb1-89e6-d448125d94f3	1987-12-12: Hampton Coliseum, Hampton, VA, USA	197	2128729	3	\N	120	28	\N		0	-1	2019-03-15 09:46:50.446876+00
            
# Release Group
# 2128729	f979a1c6-b6c2-4aad-b5b6-709c6c216752	1987-12-12: Hampton Coliseum, Hampton, VA, USA	197	1		0	2019-03-15 09:46:45.458509+00

# Recording
# 24365864	6a355a78-06c0-4e06-a15e-05d6e533255e	Sunday Bloody Sunday	197	366000		0	2019-03-15 12:29:22.049215+00	f

# Track
# 27710495	fc969b10-fdba-4b0c-82d9-aa6a7179e41f	24365864	2571008	7	7	Sunday Bloody Sunday	197	366000	0	2019-03-15 12:29:22.049215+00	f
ts = timestat("Loading Recording Data")
files = glob("mbdump/track")
trackData = {fileInfo(ifile).basename: loadData(ifile) for ifile in files}
print("Keys: {0}".format(trackData.keys()))
trackData = {key: val[list(colnames[key].keys())].rename(columns=colnames[key]) for key,val in trackData.items() if key in colnames} if colnames is not None else trackData
print("Keys: {0}".format(trackData.keys()))
ts.stop()

# Release

In [ ]:
colnames = {}
#colnames["release_alias_type"] = {0: "ReleaseAliasTypeID", 1: "ReleaseAliasTypeName"}
#colnames["release_alias"] = {0: "ReleaseAliasID", 1: "ReleaseID", 2: "ReleaseAliasName", 3: "ReleaseAliasLang", 7: "ReleaseAliasName2"}
colnames["release_status"] = {0: "ReleaseStatusID", 1: "ReleaseStatusName", 3: "NA3", 3: "ReleaseStatusDescr", 4: "ReleaseStatusGID"}
colnames["release_packaging"] = {0: "ReleasePackagingID", 1: "ReleasePackagingName", 3: "NA3", 4: "ReleasePackagingDescr", 5: "ReleaseStatusGID"}
colnames["release_label"] = {0: "Index", 1: "ReleaseID", 2: "LabelID", 3: "CatalogNumber"}
colnames["release_country"] = {0: "ReleaseID", 1: "ReleaseCountryID", 2: "ReleaseCountryYear", 3: "PossibleHash3", 4: "PossibleHash4"}
colnames["release_unknown_country"] = {0: "ReleaseID", 1: "ReleaseCountryYear", 2: "NA2", 3: "NA3"}
colnames["release"] = {0: "ReleaseID", 1: "ReleaseGID", 2: "ReleaseName", 3: "ArtistID", 4: "ReleaseGroupID", 5: "ReleaseStatusID", 
                       6: "ReleasePackagingID", 7: "LanguageID", 8: "ScriptID", 9: "ReleaseBarcode", 10: "ReleaseComment", 11: "NA11", 12: "NA12"}

ts = timestat("Loading Release Data")
files = [ifile for ifile in glob("mbdump/release*") if "group" not in ifile]
releaseData = {fileInfo(ifile).basename: loadData(ifile) for ifile in files}
print("Keys: {0}".format(releaseData.keys()))
releaseData = {key: val[list(colnames[key].keys())].rename(columns=colnames[key]) for key,val in releaseData.items() if key in colnames} if colnames is not None else releaseData
print("Keys: {0}".format(releaseData.keys()))
ts.stop()

## Indexing

In [ ]:
ts = timestat("Setting Index For Release DataFrames")
for key,df in releaseData.items():
    colname = df.columns[0]
    df.index = df[colname]
    df.drop([colname], axis=1, inplace=True)
ts.stop()

In [ ]:
releaseData['release'].head()

In [ ]:
releaseData['release_packaging']

In [ ]:
# Peter Gabriel , II : 133050
# Pixies , Velouria  : 57759
pixiesVelouriaList = releaseData['release'][releaseData['release']['ReleaseGroupID'] == 57759].index
releaseData['release'][releaseData['release']['ReleaseGroupID'] == 57759]

In [ ]:
releaseData['release'][releaseData['release']['NA11'] > 0.0]

In [ ]:
key = "release"
col = "NA11"
print("Min:",releaseData[key][col][releaseData[key][col].notna()].astype(int).min())
print("Max:",releaseData[key][col][releaseData[key][col].notna()].astype(int).max())
print("Uni:",releaseData[key][col][releaseData[key][col].notna()].astype(int).nunique())
releaseData[key][col][releaseData[key][col].notna()].astype(int).value_counts()

In [ ]:
key = "release"
col = "NA12"
print("Min:",releaseData[key][col][releaseData[key][col].notna()].astype(int).min())
print("Max:",releaseData[key][col][releaseData[key][col].notna()].astype(int).max())
print("Uni:",releaseData[key][col][releaseData[key][col].notna()].astype(int).nunique())
releaseData[key][col][releaseData[key][col].notna()].astype(int).value_counts()

In [ ]:
releaseData['release_country'][releaseData['release_country'].index.isin(releaseIDs)]

In [ ]:
releaseData['release_label'][releaseData['release_label']["ReleaseID"].isin(releaseIDs)]

In [ ]:
releaseData['release_label'][releaseData['release_label'].index == 1573831]

In [ ]:
print("Min:",releaseData['release']["NA8"][releaseData['release']["NA8"].notna()].astype(int).min())
print("Max:",releaseData['release']["NA8"][releaseData['release']["NA8"].notna()].astype(int).max())
print("Uni:",releaseData['release']["NA8"][releaseData['release']["NA8"].notna()].astype(int).nunique())
releaseData['release']["NA8"][releaseData['release']["NA8"].notna()].astype(int).value_counts()

In [ ]:
# "https://musicbrainz.org/release-group/6c301dbd-6ccb-3403-a6c4-6a22240a0297
# "U2": "The Best of 1980–1990" Album + Complication
releaseGroupID  = 564345
releaseGroupGID = 'c301dbd-6ccb-3403-a6c4-6a22240a0297'

In [ ]:
releaseIDs = releaseData['release'][releaseData['release']['ReleaseGroupID'] == releaseGroupID].index
releaseIDs

In [ ]:
releaseData['release'][releaseData['release']['ReleaseGroupID'] == releaseGroupID]

In [ ]:
releaseData['release'][(releaseData['release']['ReleaseGroupID'] == releaseGroupID) & (releaseData['release'].index == 2052110)]

In [ ]:
colnames["release_country"] = {0: "ReleaseID", 1: "ReleaseCountryID", 2: "ReleaseCountryYear", 3: "NA3", 4: "NA4"}
colnames["release_unknown_country"] = {0: "ReleaseID", 1: "ReleaseCountryYear", 2: "NA2", 3: "NA3"}


In [ ]:
releaseData['release_country'][releaseData['release_country'].index.isin([1321245,  670731,  210217, 1596225,  209665, 2046253, 2052110,
             132340, 1828668, 2221126, 2472174, 2503322, 2599439, 2791379,
             209663, 2780403, 2830545, 2880210, 2881926])]

In [ ]:
releaseData['release_country'][releaseData['release_country'].index == 2830545]

## Extra

In [ ]:
artistIDNumRelease = DataFrame(releaseData['release']["ArtistID"].value_counts())
artistIDNumRelease.columns = ["NumReleases"]
artistIDNumRelease.head()

In [ ]:
releaseData['release']

# Release-Group

In [ ]:
colnames = {}
#colnames["release_group_alias_type"] = {0: "ReleaseGroupAliasTypeID", 1: "ReleaseGroupAliasTypeName"}
#colnames["release_group_alias"] = {0: "ReleaseGroupAliasID", 1: "NA1", 2: "ReleaseGroupAliasName", 3: "ReleaseGroupAliasLang", 7: "ReleaseGroupAliasName2"}
#colnames["release_group_primary_type"] = {0: "ReleaseGroupPrimaryTypeID", 1: "ReleaseGroupPrimaryTypeName", 3: "NA3"}
#colnames["release_group_secondary_type"] = {0: "ReleaseGroupSecondaryTypeID", 1: "ReleaseGroupSecondaryTypeName"}
colnames["release_group"] = {0: "ReleaseGroupID", 1: "ReleaseGroupGID", 2: "ReleaseGroupName", 3: "ArtistID", 4: "ReleaseGroupPrimaryTypeID", 5: "ReleaseGroupComment", 6: "NA6"}
colnames["release_group_secondary_type_join"] = {0: "ReleaseGroupID", 1: "ReleaseGroupSecondaryTypeID"}

ts = timestat("Loading Release Data")
files = [ifile for ifile in glob("mbdump/release*") if "group" in ifile]
releaseGroupData = {fileInfo(ifile).basename: loadData(ifile) for ifile in files}
print("Keys: {0}".format(releaseGroupData.keys()))
releaseGroupData = {key: val[list(colnames[key].keys())].rename(columns=colnames[key]) for key,val in releaseGroupData.items() if key in colnames} if colnames is not None else releaseGroupData
print("Keys: {0}".format(releaseGroupData.keys()))
ts.stop()

## Indexing

In [ ]:
ts = timestat("Setting Index For ReleaseGroup DataFrames")
for key,df in releaseGroupData.items():
    colname = df.columns[0]
    df.index = df[colname]
    df.drop([colname], axis=1, inplace=True)
ts.stop()

## Joins

In [ ]:
ts = timestat("Joining Secondary Group Type To Master Data")
masterReleaseGroupData = releaseGroupData['release_group'].join(releaseGroupData['release_group_secondary_type_join'])
ts.update()
masterReleaseGroupData["ReleaseGroupPrimaryTypeID"] = masterReleaseGroupData["ReleaseGroupPrimaryTypeID"].fillna(-1)
masterReleaseGroupData["ReleaseGroupPrimaryTypeID"] = masterReleaseGroupData["ReleaseGroupPrimaryTypeID"].astype(int)

masterReleaseGroupData["ReleaseGroupSecondaryTypeID"] = masterReleaseGroupData["ReleaseGroupSecondaryTypeID"].fillna(-1)
masterReleaseGroupData["ReleaseGroupSecondaryTypeID"] = masterReleaseGroupData["ReleaseGroupSecondaryTypeID"].astype(int)
ts.update()
masterReleaseGroupData["ReleaseGroupTypeKey"] = tuple(zip(masterReleaseGroupData["ReleaseGroupPrimaryTypeID"], masterReleaseGroupData["ReleaseGroupSecondaryTypeID"]))
ts.stop()

## Create Artist/ReleaseGroupType Map

In [ ]:
masterSaveData = masterReleaseGroupData[["ArtistID", "ReleaseGroupTypeKey", "ReleaseGroupName"]]
from fileIO import pickleIO
io = pickleIO()
for modVal in range(10):
    ts = timestat("Creating Artist/Release Data For ModVal={0}".format(modVal))
    modValData = {artistID: {groupTypeKey: groupTypeKeyData["ReleaseGroupName"].to_dict() for groupTypeKey,groupTypeKeyData in artistDF.groupby("ReleaseGroupTypeKey")} for artistID,artistDF in masterSaveData[masterSaveData["ArtistID"] % 10 == modVal].groupby("ArtistID")}
    ts.update()
    io.save(idata=modValData, ifile="mbdump/MasterArtistReleaseGroupDataMod{0}.p".format(modVal))
    ts.stop()
    print("")

## Extra

In [ ]:
releaseGroupData['release_group'][(releaseGroupData['release_group']['ArtistID'] == 197) & (releaseGroupData['release_group']['ReleaseGroupName'] == "Previous Day")]

In [ ]:
releaseGroupData['release_group'][releaseGroupData['release_group']["ReleaseGroupGID"] == "744c7a1b-ac79-35c4-bd92-7e2c6a24c8d8"]

In [ ]:
files = glob("mbdump/release_group")
files = [ifile for ifile in glob("mbdump/release*") if "group" in ifile]
releaseGroupData = {fileInfo(ifile).basename: loadData(ifile) for ifile in files}

In [ ]:
releaseGroupData['release_group']["ReleaseGroupPrimaryType"].nunique()

# Artist

In [ ]:
colnames = {}
colnames["artist_credit"] = {0: "ArtistCreditID", 1: "ArtistCreditName", 2: "ArtistCreditNum", 3: "NA3"}
colnames["artist_type"] = {0: "ArtistTypeID", 1: "ArtistTypeName", 2: "NA2", 3: "NA3", 4: "ArtistTypeDescr", 5: "ArtistTypeGID"}
colnames["artist_isni"] = {0: "ArtistISNIID", 1: "ArtistISNI"}
colnames["artist_alias_type"] = {0: "ArtistAliasTypeID", 1: "ArtistAliasTypeName", 5: "ArtistAliasTypeGID"}
colnames["artist_alias"] = {0: "ArtistAliasID", 1: "ArtistID", 2: "ArtistAliasName", 3: "ArtistAliasLang", 7: "ArtistAliasSortName"}
colnames["artist"] = {0: "ArtistID", 1: "ArtistGID", 2: "ArtistName", 3: "ArtistSortName",
                      4: "FormedYear", 5: "FormedMonth", 6: "FormedDay", 
                      7: "DisbandedYear", 8: "DisbandedMonth", 9: "DisbandedDay", 
                      10: "ArtistTypeID", 11: "CountryAreaID", 12: "GenderID", 13: "ArtistDescr", 14: "NA14", 17: "FoundedInAreaID", 18: "DisbandedInAreaID"}

ts = timestat("Loading Artist Data")
files = glob("mbdump/artist*")
artistData = {fileInfo(ifile).basename: loadData(ifile) for ifile in files if ifile not in ["mbdump/artist_credit_name"]}
print("Keys: {0}".format(artistData.keys()))
artistData = {key: val[list(colnames[key].keys())].rename(columns=colnames[key]) for key,val in artistData.items() if key in colnames} if colnames is not None else artistData
print("Keys: {0}".format(artistData.keys()))
ts.stop()

In [ ]:
artistData['artist'][artistData['artist']['NA14'] == 23]

In [ ]:
artistData['artist'].index = artistData['artist']['ArtistID'].to_list()
artistData['artist'].drop(['ArtistID'], axis=1, inplace=True)

In [ ]:
artistData['artist']

# Master Artist Summary Data

In [ ]:
masterArtistData = artistData['artist'][["ArtistGID", "ArtistName", "ArtistSortName", "Formed", "Disbanded"]].copy(deep=True)
masterArtistNumAlbums = artistIDNumReleaseGroups.join(artistIDNumRelease, how='outer')
masterArtistData = masterArtistData.join(masterArtistNumAlbums)
masterArtistData["NumReleaseGroups"] = masterArtistData["NumReleaseGroups"].fillna(0).apply(int)
masterArtistData["NumReleases"] = masterArtistData["NumReleases"].fillna(0).apply(int)

In [ ]:
masterArtistData

In [ ]:
masterartistNumAlbums

In [ ]:
artistIDNumReleaseGroups.shape

In [ ]:
artistIDNumRelease.shape

In [ ]:
artistData['artist'][artistData['artist']['ArtistID'] == aIDs["ArianaGrande"]]

In [ ]:
artistData['artist'][artistData['artist']['ArtistID'] == aIDs["BuddyHolly"]]

In [ ]:
artistData['artist'][artistData['artist']['ArtistID'] == aIDs["Bono"]]

In [ ]:
artistData['artist'][artistData['artist']['ArtistID'] == aIDs["Rupaul"]]

In [ ]:
artistData['artist'][artistData['artist']['ArtistID'] == aIDs["U2"]]

In [ ]:
artistData['artist'][artistData['artist']['ArtistID'] == aIDs["DMB"]]

In [ ]:
artistData['artist']["NA18"].value_counts()

In [ ]:
artistData['artist']["NA9"].value_counts()

In [ ]:
artistData['artist']['NA12'].unique()

In [ ]:
artistData['artist']['NA5'].unique()

In [ ]:
artistData['artist'][artistData['artist']['ArtistGID'] == '7f347782-eb14-40c3-98e2-17b6e1bfe56c']

In [ ]:
artistData["artist"][artistData["artist"]["ArtistID"] == 502]

In [ ]:
artistData["artist"]['NA10'].unique()

In [ ]:
artistData["artist"][artistData["artist"]["ArtistID"] == 197]

# Artist Lookup

In [ ]:
colnames["l_artist_url"]={0: "ArtistURLLID", 1: "URLGroupID", 2: "ArtistID", 3: "URLID"}
colnames["l_artist_release_group"]={0: "ArtistReleaseGroupLID", 1: "ReleaseGroupGroupID", 2: "ArtistID", 3: "ReleaseGroupID"}
colnames["l_artist_release"]={0: "ArtistReleaseLID", 1: "ReleaseGroupID", 2: "ArtistID", 3: "ReleaseID"}

ts = timestat("Loading Artist Data")
files = glob("mbdump/l_artist_*")
lookupData = {fileInfo(ifile).basename: loadData(ifile) for ifile in files}
lookupData = {key: val[list(colnames[key].keys())].rename(columns=colnames[key]) for key,val in lookupData.items() if key in colnames} if lookupData is not None else lookupData
print("Keys: {0}".format(lookupData.keys()))
ts.stop()

In [ ]:
files = glob("mbdump/l_artist_release")
lookupData = {fileInfo(ifile).basename: loadData(ifile) for ifile in files}

In [ ]:
lookupData['l_artist_release']["ReleaseGroupID"].nunique()

In [ ]:
lookupData['l_artist_release'][lookupData['l_artist_release']['ArtistID'] == 502]

In [ ]:
key='l_artist_url'
lookupData['l_artist_url'] = lookupData['l_artist_url'][list(colnames[key].keys())].rename(columns=colnames[key])

In [ ]:
print(urlData['url'][urlData['url']["URLName"].eq("https://www.discogs.com/artist/6520")])
print(urlData['url'][urlData['url']["URLName"].eq("https://www.allmusic.com/artist/mn0000219203")])

In [ ]:
lookupData["l_artist_url"][lookupData["l_artist_url"]["URLID"].isin([3017,993955])]

In [ ]:
from pandas import merge
dmbAU = lookupData["l_artist_url"][lookupData["l_artist_url"]['ArtistID'] == 502].copy(deep=True)
u2AU  = lookupData["l_artist_url"][lookupData["l_artist_url"]['ArtistID'] == 197].copy(deep=True)

In [ ]:
dmbURLs = merge(dmbAU, urlData['url'], how='left', on=["URLID"]).copy(deep=True)
u2URLs  = merge(u2AU, urlData['url'], how='left', on=["URLID"]).copy(deep=True)

In [ ]:
dmbURLs["URLDomain"] = dmbURLs["URLName"].apply(lambda x: x.replace("https://", "").replace("http://", "").split('/')[0])

In [ ]:
u2URLs["URLDomain"] = u2URLs["URLName"].apply(lambda x: x.replace("https://", "").replace("http://", "").split('/')[0])

In [ ]:
u2URLs[["NA1", "URLDomain"]].sort_values(by="NA1").T

In [ ]:
dmbURLs[["NA1", "URLDomain"]].sort_values(by="NA1").T

In [ ]:
artistData["artist"][artistData["artist"].eq(8723).any(1)]

In [ ]:
urlData['url'].shape

In [ ]:
lookupData["l_artist_url"][lookupData["l_artist_url"].eq(1025971).any(1)]

In [ ]:
lookupData["l_artist_url"][lookupData["l_artist_url"].eq(2625).any(1)]

In [ ]:
lookupData["l_artist_url"][lookupData["l_artist_url"][2].eq(502).any(1)]

In [ ]:
DMB={AllMusic = 1025971 (c94225e3-2f0c-4c6d-9115-9f268fb7c31b), Discogs = 2625 (7a157b6e-d01d-4248-9995-edb05652c5b2)}

In [ ]:
artistData['artist']

In [ ]:
colnames = {0: "ArtistID", 1: "NA1", 2: "NA2": 3: "NA3"}
lookupData["l_artist_artist"][lookupData["l_artist_artist"].eq(502).any(1)]

In [ ]:
urlData['url'][urlData['url']["URLName"].eq("https://www.discogs.com/artist/6520")]
urlData['url'][urlData['url']["URLName"].eq("https://www.allmusic.com/artist/mn0000219203")]

In [ ]:
502
07e748f1-075e-428d-85dc-ce3be434e906